https://github.com/OpenDataDE/State-zip-code-GeoJSON

In [1]:
import pandas as pd

In [2]:
import json
import os

In [3]:
SELECTED_NAME = "RDC"

In [4]:
geojson_dir = r"D:\GIT\zipmaps"
geojson = {
    "DC":"dc_district_of_columbia_zip_codes_geo.min.json",
    "VA":"va_virginia_zip_codes_geo.min.json",
    "MD":"md_maryland_zip_codes_geo.min.json",
}

In [5]:
CAPTURING_ZONES = 3

In [6]:
SELECTED = "22204"

In [7]:
tempMAPDATA = {
    "ZONE1":"",
    "ZONE2":"",
    "ZONE3":"",
    "ZONE4":"",
    "ZONE5":"",
    "ZONE6":"",
}

In [8]:
tempMAPDATA["ZONE1"] = """
22041, 22044, 22201, 22202, 22203, 22204, 22205, 22206, 22209, 22211, 22214, 22302, 22305, 22311
"""

In [9]:
tempMAPDATA["ZONE2"] = """
20001, 20003, 20004, 20005, 20006, 20007, 20008, 20009, 20016, 20024, 20026, 20032, 20036, 20037, 20045, 20052, 20057, 20260, 20319, 20373, 20374, 20390, 20391, 20398, 20500, 20535, 22042, 22043, 22046, 22207, 22213, 22301, 22303, 22304, 22310, 22312, 22314, 22331, 22332
"""

In [10]:
tempMAPDATA["ZONE3"] = """
20002, 20010, 20011, 20012, 20015, 20017, 20018, 20019, 20020, 20064, 20317, 20745, 20748, 20812, 20815, 20816, 20818, 22003, 22027, 22031, 22067, 22081, 22101, 22102, 22150, 22151, 22159, 22180, 22306, 22315
"""

In [11]:
tempMAPDATA["ZONE4"] = """
20710, 20712, 20722, 20737, 20742, 20743, 20746, 20747, 20781, 20782, 20783, 20814, 20817, 20895, 20901, 20910, 20912, 22015, 22032, 22060, 22152, 22153, 22181, 22182, 22309
"""

In [12]:
tempMAPDATA["ZONE5"] = """
20190, 20191, 20706, 20740, 20770, 20784, 20785, 20852, 20854, 20902, 20903, 22030, 22035, 22066, 22124
"""

In [13]:
tempMAPDATA["ZONE6"] = """
20170, 20171, 20194, 22033
"""

In [14]:
if os.path.isfile(os.path.join(geojson_dir, "delivery.json")):
    with open(os.path.join(geojson_dir, "delivery.json"), "r") as delivery_file:
        delivery_dict = json.load(delivery_file)
else:
    delivery_dict = {}

In [15]:
MAPDATA = None
if SELECTED in delivery_dict:
    if len(delivery_dict[SELECTED])>=CAPTURING_ZONES:
        MAPDATA = delivery_dict[SELECTED]["MAPDATA"]        

if not MAPDATA:
    MAPDATA = {k:v for (k,v) in tempMAPDATA.items() if v.strip()}

    MAPDATA = {k:[i.strip() for i in v.split(",")] for (k,v) in MAPDATA.items()}

    delivery_dict[SELECTED] = {}
    delivery_dict[SELECTED]["MAPDATA"] = MAPDATA
    delivery_dict[SELECTED]["SELECTED"] = SELECTED
    with open(os.path.join(geojson_dir, "delivery.json"), "w+") as delivery_file:
        delivery_file.write(json.dumps(delivery_dict, indent=4, sort_keys=True))

In [16]:
dfdata = [[k,i,index+1] for index, (k,v) in enumerate(MAPDATA.items()) for i in v]

In [17]:
dfdata = [i for i in dfdata if i[2]<=CAPTURING_ZONES]

In [18]:
selectedDF = pd.DataFrame([[SELECTED_NAME, SELECTED, 0]], columns=["ZONE", "ZIPCODE", "ZONENO"])

In [19]:
zipDF = pd.DataFrame(dfdata, columns=["ZONE", "ZIPCODE", "ZONENO"])

In [20]:
zipDF = zipDF[zipDF["ZIPCODE"] != SELECTED]

In [21]:
zipDF = selectedDF.append(zipDF, ignore_index = True)

In [22]:
geojson_data = {k:json.load(open(os.path.join(geojson_dir, v), 'r')) for (k,v) in geojson.items()}

In [23]:
combined_geojson = {'type': 'FeatureCollection', 'features': []}
for state, data in geojson_data.items():
    combined_geojson['features'].extend(data['features'])

In [24]:
## filter combined_geojson
combined_geojson['features'] = [i for i in combined_geojson['features'] if i["properties"]["ZCTA5CE10"] in list(zipDF["ZIPCODE"])]

In [25]:
import folium

In [26]:
m = folium.Map(location=[38.9,-77.2], zoom_start=11)

folium.Choropleth(
    geo_data=combined_geojson,
    fill_opacity=.7,
    line_opacity=.2,
    data=zipDF,
    columns=["ZIPCODE", "ZONENO"],
    key_on='feature.properties.ZCTA5CE10',
    fill_color='YlOrRd',
    highlight=True
).add_to(m)

for feature in combined_geojson['features']:
    c_point = (float(feature["properties"]["INTPTLAT10"]), float(feature["properties"]["INTPTLON10"]))
    folium.Marker(c_point, popup=feature["properties"]["ZCTA5CE10"]).add_to(m)

m.save(os.path.join(geojson_dir, '{}_ZipMap.html'.format(SELECTED)))